In [2]:
# -----------------------------------------------------------------------------------
# 0.  Notebook boiler-plate  ────────────────────────────────────────────────────────
# -----------------------------------------------------------------------------------
!pip install --quiet requests lxml tqdm pandas

import json, re, time, requests, os, gzip
import camelot
import pandas as pd
import glob
import pdfplumber # Import pdfplumber
import pandas as pd, textwrap, os, sys, pprint, itertools
from pathlib import Path
from datetime import datetime
from typing import List, Dict
from lxml import etree        # robust XML parser
from tqdm import tqdm # progress-bar in Jupyter
from pebble import ProcessPool
from concurrent.futures import TimeoutError as PebbleTimeoutError
import docx 


# --- Configuration: Keywords and Regular Expressions ---
FORWARD_KEYWORDS = ['forward', 'fw', 'fp', 'fwd', 'sense', 'forw']
REVERSE_KEYWORDS = ['reverse', 'rev', 'rp', 'rv', 'antisense', 'revs']
GENE_HEADER_KEYWORDS = ['gene', 'target', 'name', 'symbol', 'oligonucleotide', 'oligos', 'locus']
ALLOWED_DNA_CHARS_FOR_FILTER = "ACTG"


# -----------------------------------------------------------------------------------
# 0.5  abhi, change htis directoy here───────────────────────────────────────────────
# -----------------------------------------------------------------------------------
DATA_DIR = Path("data/psc")
DATA_DIR.mkdir(parents=True, exist_ok=True)
OUT_DIR = Path("data/psc/output")
OUT_DIR.mkdir(parents=True, exist_ok=True)


# Europe PMC imposes a 30 req/min soft limit → be polite.
HEADERS   = {"User-Agent": "PrimerMiner/0.1 (tmolley@ucsd.edu)"}
BASE_URL  = "https://www.ebi.ac.uk/europepmc/webservices/rest/"


# -----------------------------------------------------------------------------------
# 1.  Search Europe PMC for the latest 10 OA papers mentioning “GAPDH”  ─────────────
# -----------------------------------------------------------------------------------
def cleanup_downloaded_files(dirs_to_clean: List[Path]):
    """
    Deletes all files within a list of specified directories.
    This is used to manage storage space between batch runs.
    """
    for directory in dirs_to_clean:
        if not directory.is_dir():
            print(f"Cleanup skipped: Directory '{directory}' does not exist.")
            continue

        print(f"  Cleaning files in: {directory}")
        files_deleted = 0
        for item in directory.iterdir():
            # Ensure we only delete files, not subdirectories
            if item.is_file():
                try:
                    item.unlink()
                    files_deleted += 1
                except Exception as e:
                    print(f"    Could not delete file {item}: {e}")
        print(f"    ...deleted {files_deleted} file(s).")
        
def europepmc_search_all(query: str,
                         max_records: int = 10000000,
                         page_size: int = 1000,
                         throttle: float = 0.3) -> List[Dict]:
    """
    Fetch up to `max_records` hits for `query`, paging with Europe PMC's
    cursorMark API. Returns a list of JSON result dicts.
    This version is updated to handle cases where no 'nextCursorMark' is returned.
    """
    hits, retrieved = [], 0
    cursor = "*"

    while retrieved < max_records:
        url = (f"{BASE_URL}search"
               f"?query={requests.utils.quote(query)}"
               f"&format=json"
               f"&pageSize={page_size}"
               f"&cursorMark={cursor}")
        r = requests.get(url, headers=HEADERS, timeout=40)
        r.raise_for_status()
        data = r.json()

        if "resultList" not in data:
            raise RuntimeError(f"Europe PMC API error: {data.get('error', data)}")

        batch = data["resultList"]["result"]
        if not batch:  # no more results
            break

        hits.extend(batch)
        retrieved += len(batch)
        
        # --- MODIFICATION ---
        # Gracefully handle the end of results when 'nextCursorMark' is absent.
        # The .get() method returns None if the key is not found, preventing a KeyError.
        cursor = data.get("nextCursorMark")
        if not cursor:
            # No more pages to fetch, so we exit the loop.
            break
        # --- END MODIFICATION ---

        time.sleep(throttle)  # be polite (≤ 30 req / min)

    return hits[:max_records]
    
def load_hgnc_symbol_set(cache_dir="hgnc_cache") -> set[str]:
    # Use Path directly instead of pathlib.Path
    cache_path = Path(cache_dir) / "homo_sapiens.gene_info.gz"
    if not cache_path.exists():
        cache_path.parent.mkdir(parents=True, exist_ok=True)
        url = ("https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/"
               "Mammalia/Homo_sapiens.gene_info.gz")
        print("⏬  downloading HGNC symbol list …") # Corrected emoji display if needed
        try:
            r = requests.get(url, timeout=60)
            r.raise_for_status()
            cache_path.write_bytes(r.content)
        except requests.exceptions.RequestException as e:
            print(f"Error downloading HGNC symbols: {e}")
            print("Proceeding without HGNC symbol validation for XML gene names.")
            return set() # Return an empty set if download fails

    symbols = set()
    try:
        with gzip.open(cache_path, "rt", encoding='utf-8') as fh: # Added encoding
            for line in fh:
                if line.startswith("#"):
                    continue
                cols = line.rstrip("\n").split("\t")
                if len(cols) > 2: # Ensure there are enough columns
                    symbols.add(cols[2].upper())  # official symbol column
    except Exception as e:
        print(f"Error reading or processing HGNC file {cache_path}: {e}")
        print("Proceeding without HGNC symbol validation for XML gene names.")
        return set() # Return an empty set if file processing fails
    
    if not symbols:
        print(f"Warning: HGNC symbol set is empty after attempting to load from {cache_path}.")
    else:
        print(f"Loaded {len(symbols)} HGNC symbols.")
    return symbols
    
# ---------------------------------------------------------------------------
# 3  Helper: fetch XML only if we don't have it already
# ---------------------------------------------------------------------------
def get_or_download_xml(pmcid: str, dest_dir: Path) -> Path:
    """
    Return the path to the XML file for `pmcid`.
    If the file is already on disk, just return the path.
    Otherwise download from Europe PMC and save it.
    """
    fpath = dest_dir / f"{pmcid}.xml"
    if fpath.exists():                # ✅ cached
        return fpath

    # 🠞 not cached → download
    url = f"{BASE_URL}{pmcid}/fullTextXML"
    r   = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    fpath.write_text(r.text, encoding="utf-8")
    return fpath


# -----------------------------------------------------------------------------------
# 4.  Download full-text XML and supplement for each PMCID  ─────────────────────────
# -----------------------------------------------------------------------------------
def fetch_fulltext_xml(pmcid: str) -> str:
    """Return the raw JATS XML for a given PMCID from Europe PMC."""
    url = f"{BASE_URL}{pmcid}/fullTextXML"
    r   = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text

def save_xml(pmcid: str, xml_text: str) -> Path:
    """Persist the XML to disk and return the file path."""
    fpath = DATA_DIR / f"{pmcid}.xml"
    fpath.write_text(xml_text, encoding="utf-8")
    return fpath
    
def safe_json(resp):
    """Return resp.json() or an empty dict if the payload isn’t JSON."""
    try:
        if resp.text.strip():
            return resp.json()
    except json.JSONDecodeError:
        pass
    return {}
    
def rename_supp_files(pmcid: str, file_paths: list[Path]) -> list[Path]:
    """
    Rename each Path in `file_paths` to <pmcid>_supp<N>.<ext>
    Returns a list of new Path objects (same length/order as input).
    """
    new_paths = []
    for i, old_path in enumerate(sorted(file_paths), 1):
        new_name = f"{pmcid}_supp{i}{old_path.suffix.lower()}"
        new_path = old_path.with_name(new_name)
        if new_path.exists():            # avoid accidental overwrite
            new_path = old_path          # keep original
        else:
            old_path.rename(new_path)
        new_paths.append(new_path)
    return new_paths
    
def fetch_supplements_from_xml(pmcid: str,
                               xml_path: Path,
                               dest_dir: Path,
                               pause: float = 0.2) -> list[Path]:
    """
    Download *only* PDF supplementary files referenced in the JATS XML.
    """
    dest_dir.mkdir(parents=True, exist_ok=True)
    if not xml_path.exists():
        print(f"⚠️  XML file missing for {pmcid}")
        return []

    # ---- 1. gather every xlink:href ----------------------------------------
    root = etree.parse(str(xml_path))
    ns   = {"x": "http://www.w3.org/1999/xlink"}
    hrefs = set()
    hrefs.update(root.xpath("//supplementary-material/@x:href", namespaces=ns))
    hrefs.update(root.xpath("//supplementary-material//media/@x:href", namespaces=ns))
    hrefs.update(root.xpath("//ext-link[contains(@ext-link-type,'supplement')]/@x:href",
                            namespaces=ns))

    if not hrefs:
        return []

    # ---- 2. download PDF files only ---------------------------------------
    base  = f"https://europepmc.org/articles/{pmcid}/bin/"
    saved = []

    for href in hrefs:
        url   = href if href.startswith("http") else base + href.lstrip("/")
        fname = url.split("/")[-1].split("?")[0]

        # **PDF filter**
        if not fname.lower().endswith((".pdf", ".docx")):
            continue

        fpath = dest_dir / fname
        if fpath.exists():
            saved.append(fpath)
            continue

        try:
            r = requests.get(url, headers=HEADERS, timeout=60)
            r.raise_for_status()
            fpath.write_bytes(r.content)
            saved.append(fpath)
            time.sleep(pause)
        except requests.HTTPError as e:
            print(f"⚠️  {pmcid}  {fname} →", e)

    return saved
# -----------------------------------------------------------------------------------
# 5.  supplmental reader functions  ────────────────────────────────────────────────-
# -----------------------------------------------------------------------------------

def parse_docx_table_to_dataframe(docx_table_obj):
    """Converts a python-docx table object to a pandas DataFrame."""
    data = []
    keys = None
    
    # Attempt to use the first row as headers
    if docx_table_obj.rows:
        potential_header_cells = [cell.text.strip() for cell in docx_table_obj.rows[0].cells]
        # Crude check: if not too many empty cells and not overly long strings, assume header
        if len(potential_header_cells) > 0 and \
           sum(1 for h in potential_header_cells if not h) < len(potential_header_cells) / 2 and \
           all(len(h) < 100 for h in potential_header_cells): # Avoid very long cell content as header
            keys = potential_header_cells
            
    for i, row in enumerate(docx_table_obj.rows):
        text = [cell.text.strip() for cell in row.cells]
        if keys and i == 0 and len(docx_table_obj.rows) > 1: # If keys were taken from first row and there are more rows
            continue # Skip adding the header row to data list
        
        # If keys were set and match current row length, create a dict for this row
        if keys and len(text) == len(keys) and (i > 0 or len(docx_table_obj.rows) == 1):
             data.append(dict(zip(keys, text)))
        else: # Fallback to list of lists if no keys or length mismatch
            data.append(text) # Will have default integer columns if this path is taken mostly

    if not data:
        return pd.DataFrame()

    try:
        df = pd.DataFrame(data)
        # If dataframe was created from list of lists and keys were determined but not used as columns yet
        if keys and not isinstance(data[0], dict) and len(keys) == df.shape[1]:
            df.columns = keys
    except Exception: # Fallback if DataFrame creation is problematic
        df = pd.DataFrame()

    # If after all that, columns are still default integers, and first data row looks like header
    if not df.empty and all(isinstance(col, int) for col in df.columns) and df.shape[0] > 0:
        first_data_row_values = [str(c).strip() for c in df.iloc[0]]
        # Simple heuristic: if first row doesn't contain typical long sequences and has text
        is_first_row_likely_header = not any(is_valid_primer_sequence(extract_sequence_and_direction_from_cell(str(c))[0] or "", 18, 40) for c in first_data_row_values) and \
                                   any(len(str(c)) > 0 for c in first_data_row_values)
        if is_first_row_likely_header and df.shape[0] > 1:
            df.columns = first_data_row_values
            df = df[1:].reset_index(drop=True)
            
    return df

def extract_primers_from_docx(docx_path: Path, pmcid: str, hgnc_symbols: set) -> List[Dict]:
    """
    Extracts primer sequences and context from DOCX files (tables and prose).
    """
    found_docx_primers = []
    if not docx_path.exists():
        return found_docx_primers

    try:
        doc = docx.Document(docx_path) # from python-docx library
    except Exception as e:
        print(f"    Error opening DOCX file {docx_path.name}: {e}")
        return found_docx_primers

    # --- 1. Process Tables in DOCX ---
    if doc.tables:
        # print(f"    Found {len(doc.tables)} table(s) in DOCX: {docx_path.name}")
        for table_idx, docx_table_obj in enumerate(doc.tables):
            # print(f"      Processing DOCX Table {table_idx+1}...")
            df_from_docx = parse_docx_table_to_dataframe(docx_table_obj)
            if not df_from_docx.empty:
                # print(f"        Parsed DOCX table to DataFrame (shape: {df_from_docx.shape})")
                # Use your existing scan_dataframe_for_primers function
                source_desc = f"DOCX Table {table_idx+1}" # Page info is not direct like PDF
                primers_from_table = scan_dataframe_for_primers(
                    df_from_docx, 
                    pmcid,
                    source_desc, 
                    docx_path.name, # Source File
                    hgnc_symbols
                )
                if primers_from_table:
                    found_docx_primers.extend(primers_from_table)
            # else:
                # print(f"      DOCX Table {table_idx+1} resulted in an empty DataFrame.")

    # --- 2. Process Prose Text (paragraphs) in DOCX ---
    # print(f"    Scanning {len(doc.paragraphs)} paragraphs in DOCX: {docx_path.name}...")
    unique_prose_blocks_docx = set()
    dna_candidate_regex_prose = re.compile(r'\b((?:[ACGT]\s*){15,50})\b', re.IGNORECASE)

    for para_idx, para in enumerate(doc.paragraphs):
        text_block = para.text.strip()
        if not text_block or len(text_block) < 20 or text_block in unique_prose_blocks_docx:
            continue
        unique_prose_blocks_docx.add(text_block)

        for match in dna_candidate_regex_prose.finditer(text_block):
            raw_dna_match_with_spaces = match.group(1)
            core_bases, direction_from_text_body, _ = \
                extract_sequence_and_direction_from_cell(raw_dna_match_with_spaces)

            if core_bases and is_valid_primer_sequence(core_bases):
                sequence = core_bases
                probable_direction = direction_from_text_body if direction_from_text_body else "Unknown"
                probable_gene = "Unknown"
                
                match_start_in_block = match.start()
                match_end_in_block = match.end()
                
                context_chars = 80
                text_before = text_block[max(0, match_start_in_block - context_chars) : match_start_in_block]
                text_after = text_block[match_end_in_block : min(len(text_block), match_end_in_block + context_chars)]
                
                if probable_direction == "Unknown":
                    if any(k in text_before.lower() or k in text_after.lower() for k in FORWARD_KEYWORDS):
                        probable_direction = "Forward"
                    elif any(k in text_before.lower() or k in text_after.lower() for k in REVERSE_KEYWORDS):
                        probable_direction = "Reverse"
                
                gene_search_context = text_before + raw_dna_match_with_spaces + text_after
                potential_gene_matches = re.findall(r'\b([A-Z][A-Za-z0-9-]{2,15})\b', gene_search_context) # Case sensitive for initial find
                best_dist = float('inf')
                common_prose_noise = {"PCR", "DNA", "RNA", "PRIMER", "FORWARD", "REVERSE", "SEQUENCE", "METHOD", "FIGURE", "TABLE"} | set(k.upper() for k in FORWARD_KEYWORDS) | set(k.upper() for k in REVERSE_KEYWORDS)

                for pg_cand_raw in potential_gene_matches:
                    pg_cand_upper = pg_cand_raw.upper()
                    # Check if it's an HGNC symbol OR just a plausible non-noise capitalized word
                    is_hgnc_or_plausible = (pg_cand_upper in hgnc_symbols) or \
                                           (len(pg_cand_raw) > 2 and pg_cand_upper not in common_prose_noise and not pg_cand_raw.islower())

                    if is_hgnc_or_plausible:
                        try:
                            for m_pg in re.finditer(re.escape(pg_cand_raw), gene_search_context): # Match original case
                                primer_rel_start = len(text_before)
                                dist = abs(m_pg.start() - primer_rel_start)
                                if dist < best_dist and dist < context_chars : 
                                    best_dist = dist
                                    cleaned_pg_cand = clean_gene_name(pg_cand_raw)
                                    if cleaned_pg_cand != "Unknown": # Ensure clean_gene_name doesn't reject it
                                        probable_gene = cleaned_pg_cand
                        except: pass 
                
                original_context_snip = f"...{text_block[max(0,match_start_in_block-30):min(len(text_block),match_end_in_block+30)]}..."
                found_docx_primers.append({
                    "PMCID": pmcid, "Gene": probable_gene, "Sequence": sequence,
                    "Orientation": probable_direction, "Source File": docx_path.name, 
                    "Page": f"DOCX Paragraph", # Can't get page number easily from python-docx paragraphs
                    "Original Cell Text": original_context_snip 
                })
    return found_docx_primers

    
def extract_pmcid_from_filename(filename):
    if not isinstance(filename, str):
        return "UnknownPMCID"
    # Regex to capture characters before "_supp" followed by anything and ".pdf"
    # This handles variations like _supp1.pdf, _supplemental.pdf, etc.
    match = re.match(r"^(.*?)_supp(?:lemental|lement|s)?\d*\.pdf$", filename, re.IGNORECASE)
    if match:
        pmcid_candidate = match.group(1)
        # Further ensure it looks like a PMCID if that's the strict requirement
        if pmcid_candidate.upper().startswith("PMC"):
            return pmcid_candidate
        # If not starting with PMC, it might be another ID format used before "_supp"
        return pmcid_candidate 
    
    # Fallback if no "_supp" pattern but filename itself is a PMCID
    pmc_direct_match = re.match(r"^(PMC\d+)\.pdf$", filename, re.IGNORECASE)
    if pmc_direct_match:
        return pmc_direct_match.group(1)

    # Generic fallback: return filename without extension if no specific pattern found
    return os.path.splitext(filename)[0]


def is_plausible_dna_segment(text_segment):
    if not isinstance(text_segment, str) or not text_segment.strip():
        return False
    allowed_interspersed_chars = " -" 
    has_dna_char = False
    for char_val in text_segment.upper():
        if char_val in ALLOWED_DNA_CHARS_FOR_FILTER:
            has_dna_char = True
        elif char_val in allowed_interspersed_chars:
            continue
        elif char_val.isalpha(): 
            return False 
    return has_dna_char

def extract_sequence_and_direction_from_cell(cell_text):
    if not isinstance(cell_text, str) or not cell_text.strip():
        return None, None, ""

    text_body = cell_text.strip()
    original_text_for_context = text_body
    inferred_direction = None
    
    # This section for identifying direction keywords (e.g., "Forward", "Rv") is preserved as it works well.
    direction_affixes = {
        "Forward": [
            (r"^(?:FP[\s:-]*|FWD[\s:-]*|FORWARD[\s:-]*|SENSE[\s:-]*)", "prefix"),
            (r"\s*\((?:Forward|FWD|FP|Sense)\)$", "suffix")
        ],
        "Reverse": [
            (r"^(?:RP[\s:-]*|REV[\s:-]*|REVERSE[\s:-]*|ANTISENSE[\s:-]*)", "prefix"),
            (r"\s*\((?:Reverse|REV|RP|Antisense)\)$", "suffix")
        ]
    }
    prime_5_prefix_pattern = r"^(?:5['`´‘’]?[-–—]?)"
    prime_3_suffix_pattern = r"(?:[-–—]?3['`´‘’]?)$"

    text_body_changed_in_pass = True
    while text_body_changed_in_pass:
        text_body_before_affix_pass = text_body
        text_body_changed_in_pass = False
        current_pass_direction_found = False
        for dir_key, patterns in direction_affixes.items():
            if current_pass_direction_found and inferred_direction: break
            for pattern_str, affix_type in patterns:
                original_len_text_body = len(text_body)
                if affix_type == "prefix":
                    match = re.match(pattern_str, text_body, re.IGNORECASE)
                    if match:
                        if not inferred_direction: inferred_direction = dir_key
                        text_body = text_body[match.end():].strip()
                        current_pass_direction_found = True
                elif affix_type == "suffix":
                    match = re.search(pattern_str, text_body, re.IGNORECASE)
                    if match:
                        if not (match.start() == 0 and inferred_direction):
                            if not inferred_direction: inferred_direction = dir_key
                            text_body = text_body[:match.start()].strip()
                
                if len(text_body) != original_len_text_body:
                    text_body_changed_in_pass = True
                    if current_pass_direction_found: break
            if current_pass_direction_found and inferred_direction: break

    text_body = re.sub(prime_5_prefix_pattern, "", text_body, flags=re.IGNORECASE).strip()
    text_body = re.sub(prime_3_suffix_pattern, "", text_body, flags=re.IGNORECASE).strip()

    # --- REPLACEMENT LOGIC ---
    # The old logic (using is_plausible_dna_segment and filter) is replaced.
    # This new regex approach is more robust and solves both of your issues.

    # 1. Find all contiguous blocks of valid DNA characters (A, C, T, G).
    #    - We set a minimum length (e.g., 10) to avoid matching short acronyms.
    #    - This cleanly separates primers from gene names ("GAPDH") and ignores junk ("â€²").
    primer_candidates = re.findall(r'([ACTG]{10,})', text_body, re.IGNORECASE)
    
    if not primer_candidates:
        return None, inferred_direction, original_text_for_context

    # 2. From the candidates found, choose the longest one that passes full validation.
    best_sequence = ""
    for candidate in primer_candidates:
        # Your 'is_valid_primer_sequence' function already checks for the correct length (e.g., 18-40).
        if is_valid_primer_sequence(candidate) and len(candidate) > len(best_sequence):
            best_sequence = candidate
    
    if best_sequence:
        # 3. Return the cleaned, validated, uppercase sequence if found.
        return best_sequence.upper(), inferred_direction, original_text_for_context
    else:
        # No candidate passed the final validation (e.g., all were too short or too long).
        return None, inferred_direction, original_text_for_context

def is_valid_primer_sequence(sequence_bases, min_len=18, max_len=40):
    if not isinstance(sequence_bases, str): return False 
    if not (min_len <= len(sequence_bases) <= max_len):
        return False
    if not all(char_val.upper() in ALLOWED_DNA_CHARS_FOR_FILTER for char_val in sequence_bases):
        return False
    return True

def clean_gene_name(gene_text):
    if not isinstance(gene_text, str) or not gene_text.strip() or gene_text.lower() == 'nan':
        return "Unknown"
    match = re.match(r"([a-zA-Z0-9_.\-]+(?:[-/][a-zA-Z0-9_.\-]+)*)", gene_text.strip())
    if match:
        name = match.group(1)
        if len(name) > 1 and not (name.isnumeric() and len(name) > 4): 
            # Convert common direction keywords found in gene cells to "Unknown" gene
            # if they were mistakenly identified as gene name
            if name.lower() not in (FORWARD_KEYWORDS + REVERSE_KEYWORDS + ["sense", "antisense", "forward primer", "reverse primer"]):
                 # Avoid names that are purely sequence like
                if not (len(name) > 15 and all(c.upper() in ALLOWED_DNA_CHARS_FOR_FILTER for c in name)):
                    return name
    return "Unknown" 


def find_primers_sequence_first(pdf_path, page_spec="all"):
    all_found_primers_details = []
    pdf_filename = os.path.basename(pdf_path)

    if not os.path.exists(pdf_path):
        print(f"Error: PDF file not found at {pdf_path}")
        return all_found_primers_details

    print(f"Attempting to read tables from '{pdf_filename}' pages: {page_spec}")
    
    start_camelot_time = time.time()
    lattice_tables_content = []
    stream_tables_content = []

    try:
        tables_lattice_obj = camelot.read_pdf(pdf_path, pages=page_spec, flavor='lattice', line_scale=30, shift_text=[' '], copy_text=['v'], suppress_stdout=True)
        if tables_lattice_obj.n > 0: 
            lattice_tables_content = list(tables_lattice_obj) 
    except Exception as e_lattice:
        print(f"  Lattice flavor failed for {pdf_filename}. Error: {e_lattice}")

    try:
        tables_stream_obj = camelot.read_pdf(pdf_path, pages=page_spec, flavor='stream', suppress_stdout=True)
        if tables_stream_obj.n > 0: 
            stream_tables_content = list(tables_stream_obj) 
    except Exception as e_stream:
        print(f"  Stream flavor failed for {pdf_filename}. Error: {e_stream}")
    
    end_camelot_time = time.time()
    print(f"  Camelot PDF processing took: {end_camelot_time - start_camelot_time:.2f} seconds.")
        
    combined_tables_list = lattice_tables_content + stream_tables_content
        
    if not combined_tables_list:
        print(f"No tables found by Camelot (lattice or stream combined) in {pdf_filename} on pages '{page_spec}'.")
        return all_found_primers_details
    # --- End Timing for Camelot ---  
    
    start_python_processing_time = time.time()
    unique_table_identifiers = set() 

    for table_idx, table_obj in enumerate(combined_tables_list): 
        try:
            table_content_hash = hash(table_obj.df.to_string())
            table_id = (table_obj.page, table_content_hash, table_obj.flavor) 
            if table_id in unique_table_identifiers: continue
            unique_table_identifiers.add(table_id)
        except Exception: pass

        df = table_obj.df 
        # print(f"\n--- Scanning Table {table_idx+1} (Page {table_obj.page}, Shape {df.shape}, Flavor {table_obj.flavor}) ---")

        if df.empty or df.shape[0] < 1 or df.shape[1] < 1: continue

        # Get column headers for context (from df.columns and potentially df.iloc[0])
        headers_from_df_columns = [str(col).strip().lower() for col in df.columns] 
        first_row_values_as_headers = [str(df.iloc[0, c_idx]).strip().lower() for c_idx in range(min(df.shape[1], len(df.iloc[0])))] if df.shape[0] > 0 else []
    
        for r_idx in range(df.shape[0]):
            # Attempt to determine a contextual gene for the entire row first
            row_context_gene = "Unknown"
            # Check first cell of the row for gene, if it's not a sequence itself
            if df.shape[1] > 0:
                first_cell_text = str(df.iloc[r_idx, 0]).strip()
                if first_cell_text and first_cell_text.lower() != 'nan':
                    temp_bases, _, _ = extract_sequence_and_direction_from_cell(first_cell_text)
                    if not (temp_bases and is_valid_primer_sequence(temp_bases, min_len=15)):
                        cleaned_name = clean_gene_name(first_cell_text)
                        if cleaned_name != "Unknown": row_context_gene = cleaned_name
            
            for c_idx in range(df.shape[1]):
                try:
                    raw_cell_text = str(df.iloc[r_idx, c_idx]).strip()
                    if not raw_cell_text or raw_cell_text.lower() == 'nan' or len(raw_cell_text) < 10: 
                        continue

                    core_bases, direction_from_cell, original_cell_text = \
                        extract_sequence_and_direction_from_cell(raw_cell_text)

                    if core_bases and is_valid_primer_sequence(core_bases):
                        probable_gene = row_context_gene # Start with row-level gene context
                        probable_direction = direction_from_cell if direction_from_cell else "Unknown"

                        # If gene is still unknown from row context, try column to the left (if not first col)
                        if probable_gene == "Unknown" and c_idx > 0:
                            gene_cand_text = str(df.iloc[r_idx, c_idx - 1]).strip()
                            if gene_cand_text and gene_cand_text.lower() != 'nan':
                                temp_bases, _, _ = extract_sequence_and_direction_from_cell(gene_cand_text)
                                if not (temp_bases and is_valid_primer_sequence(temp_bases, min_len=15)):
                                    cleaned_name = clean_gene_name(gene_cand_text)
                                    if cleaned_name != "Unknown": probable_gene = cleaned_name
                        
                        # Refine gene if current cell (which has primer) also has a gene-like prefix not caught by direction stripping
                        if probable_gene == "Unknown" or len(probable_gene) < 2 : # If context gene weak or unknown
                            gene_part_from_primer_cell_match = re.match(r"([a-zA-Z0-9_.\-]+)\s*[:\-(]", original_cell_text)
                            if gene_part_from_primer_cell_match:
                                potential_gene_prefix = clean_gene_name(gene_part_from_primer_cell_match.group(1))
                                if potential_gene_prefix != "Unknown":
                                    probable_gene = potential_gene_prefix


                        # Attempt to find Direction from column header if not found in cell:
                        if probable_direction == "Unknown":
                            current_col_header = ""
                            # Prioritize Camelot's direct column name if it's text
                            if c_idx < len(headers_from_df_columns) and not all(isinstance(c, int) for c in df.columns):
                                current_col_header = headers_from_df_columns[c_idx]
                            # Else, if columns were int, first row might be header
                            elif c_idx < len(first_row_values_as_headers): 
                                current_col_header = first_row_values_as_headers[c_idx]

                            if current_col_header: 
                                if any(k in current_col_header for k in FORWARD_KEYWORDS): probable_direction = "Forward"
                                elif any(k in current_col_header for k in REVERSE_KEYWORDS): probable_direction = "Reverse"
                        
                        # Final fallback for direction if sequence is in one cell, and gene name is in cell above, 
                        # and direction keywords are beside gene name in cell above
                        if probable_direction == "Unknown" and r_idx > 0 and probable_gene != "Unknown":
                            cell_above_text = str(df.iloc[r_idx-1, c_idx]).strip().lower()
                            if probable_gene.lower() in cell_above_text: # if gene name is part of cell above
                                if any(k in cell_above_text for k in FORWARD_KEYWORDS): probable_direction = "Forward"
                                elif any(k in cell_above_text for k in REVERSE_KEYWORDS): probable_direction = "Reverse"


                        all_found_primers_details.append({
                            "Sequence": core_bases, "Probable Gene": probable_gene, 
                            "Probable Direction": probable_direction, "Source File": pdf_filename, 
                            "Page": table_obj.page, "Original Cell Text": original_cell_text
                        })
                except Exception: continue

    
     # --- Transform to new output format (with .get() for safety) ---
    final_results_transformed = []
    for p_info in all_found_primers_details:
        # Use .get() to provide default values if a key is somehow missing from p_info
        source_file = p_info.get("Source File", pdf_filename) # Fallback to current pdf_filename
        probable_gene = p_info.get("Probable Gene", "Unknown")
        sequence_val = p_info.get("Sequence", "") # Must have sequence
        probable_direction = p_info.get("Probable Direction", "Unknown")
        page_val = p_info.get("Page", 0) # Default page to 0 if missing
        original_text = p_info.get("Original Cell Text", "")

        if not sequence_val: # Skip if no sequence was extracted for this entry
            continue

        pmcid = extract_pmcid_from_filename(source_file)
        
        orientation = "Unknown"
        if probable_direction: 
            direction_lower = probable_direction.lower()
            if "forward" in direction_lower or "sense" in direction_lower:
                orientation = "Forward"
            elif "reverse" in direction_lower or "antisense" in direction_lower:
                orientation = "Reverse"

        final_results_transformed.append({
            "PMCID": pmcid,
            "Gene": probable_gene,
            "Sequence": sequence_val,
            "Orientation": orientation,
            "Source File": source_file,
            "Page": page_val,
            "Original Cell Text": original_text
        })
    
    end_python_processing_time = time.time()
    # print(f"  Python post-Camelot processing took: {end_python_processing_time - start_python_processing_time:.2f} seconds.")
            
    return final_results_transformed

# Add this function definition with your other helper functions

# -----------------------------------------------------------------------------------
# 6.  main text reader functions  ────────────────────────────────────────────────-
# -----------------------------------------------------------------------------------
def process_pdf_with_timeout_wrapper(args_tuple):
    """
    Wrapper function to call find_primers_sequence_first for a single PDF.
    Handles arguments and exceptions for multiprocessing.
    """
    pdf_path, page_spec, hgnc_symbols_set, pdf_filename_for_log = args_tuple
    # print(f"  Worker starting for: {pdf_filename_for_log}, pages: {page_spec}") # Optional: for verbose worker start
    try:
        result = find_primers_sequence_first(pdf_path, page_spec, hgnc_symbols_set)
        # print(f"  Worker finished for: {pdf_filename_for_log}. Found {len(result)} primer items.") # Optional
        return result
    except Exception as e:
        print(f"  ERROR in worker process for {pdf_filename_for_log}: {e}")
        import traceback
        traceback.print_exc() # Print full traceback from worker for debugging
        return [] # Return empty list on error within the worker
        
def get_element_text_content(element) -> str:
    """Extracts and concatenates all text within an XML element."""
    return " ".join(text.strip() for text in element.xpath(".//text()") if text.strip()).strip()
def parse_xml_table_to_dataframe(table_etree_element):
    """Converts an lxml etree <table> element to a pandas DataFrame."""
    headers = []
    header_elements = table_etree_element.xpath("./thead/tr/th | ./tr[1]/th | ./thead/tr/td | ./tr[1]/td") # More comprehensive header find
    
    # If the first row has only <th>, it's likely the header row
    if header_elements and all(el.tag == 'th' for el in table_etree_element.xpath("./tr[1]/*")):
         headers = [get_element_text_content(th).strip() for th in header_elements]
    elif header_elements : # if first row has mix of th/td or just td but are headers from xpath
         headers = [get_element_text_content(th).strip() for th in header_elements]


    rows_data = []
    # Get all table rows (tr)
    tr_elements = table_etree_element.xpath("./tbody/tr | ./tr") # Get body rows or all rows if no tbody
    
    start_row_index = 0
    if headers and len(tr_elements) > 0 : # If headers were found from tr[1] or thead
        # Check if the first tr_element was used for headers
        first_tr_cells_for_header_check = tr_elements[0].xpath("./th | ./td")
        first_tr_text_for_header_check = [get_element_text_content(c).strip() for c in first_tr_cells_for_header_check]
        if first_tr_text_for_header_check == headers:
            start_row_index = 1 # Data rows start from the next row

    for r_idx in range(start_row_index, len(tr_elements)):
        tr = tr_elements[r_idx]
        cells = tr.xpath("./td | ./th") # Cells can be td or th (e.g. row headers)
        row_text = [get_element_text_content(cell).strip() for cell in cells]
        if any(rt.strip() for rt in row_text): # Only add non-empty rows
            rows_data.append(row_text)

    if not rows_data:
        return pd.DataFrame()

    df = pd.DataFrame(rows_data)
    if headers and len(headers) == df.shape[1]:
        df.columns = headers
    elif df.shape[0] > 0 and not headers: # No explicit headers found, try to use first data row if suitable
        first_row_is_data_like = any(is_valid_primer_sequence(extract_sequence_and_direction_from_cell(str(c))[0] or "", 18,40) for c in df.iloc[0])
        if not first_row_is_data_like and df.shape[0] > 1 :
            df.columns = [str(c).strip() for c in df.iloc[0]]
            df = df[1:].reset_index(drop=True)
        # Else: use default integer column names from pandas
    return df

def scan_dataframe_for_primers(df: pd.DataFrame, pmcid: str, 
                               source_description: str, # e.g., "XML Table in file.xml" or "PDF Page X"
                               source_filename_context: str, # filename for XML, or page number for PDF
                               hgnc_symbols: set) -> List[Dict]:
    """
    Scans a given DataFrame (from PDF or XML table) for primer sequences.
    """
    found_primers_in_df = []
    if df.empty:
        return found_primers_in_df

    df_headers_original_case = [str(col).strip() for col in df.columns] # Keep original case for display/debug if needed
    df_headers_lower = [h.lower() for h in df_headers_original_case]

    for r_idx in range(df.shape[0]):
        row_context_gene = "Unknown"
        # Try to get row context gene from the first cell if it's text-like and not a sequence
        if df.shape[1] > 0:
            first_cell_text_for_gene = str(df.iloc[r_idx, 0]).strip()
            if first_cell_text_for_gene and first_cell_text_for_gene.lower() != 'nan':
                temp_bases_gene, _, _ = extract_sequence_and_direction_from_cell(first_cell_text_for_gene)
                # A gene name shouldn't be a long valid primer sequence
                if not (temp_bases_gene and is_valid_primer_sequence(temp_bases_gene, min_len=15)): # Check if it's NOT a primer
                    cleaned_name = clean_gene_name(first_cell_text_for_gene) # clean_gene_name does further checks
                    if cleaned_name != "Unknown":
                         # Validate against HGNC if it looks like a gene symbol
                        if cleaned_name.upper() in hgnc_symbols or (len(cleaned_name) > 4 and not any(k in cleaned_name.lower() for k in FORWARD_KEYWORDS + REVERSE_KEYWORDS)): # Allow longer non-HGNC names
                            row_context_gene = cleaned_name

        for c_idx in range(df.shape[1]):
            try:
                raw_cell_text = str(df.iloc[r_idx, c_idx]).strip()
                if not raw_cell_text or raw_cell_text.lower() == 'nan' or len(raw_cell_text) < 10:
                    continue

                core_bases, direction_from_cell, original_cell_text = \
                    extract_sequence_and_direction_from_cell(raw_cell_text)

                if core_bases and is_valid_primer_sequence(core_bases):
                    sequence = core_bases
                    probable_direction = direction_from_cell if direction_from_cell else "Unknown"
                    probable_gene = row_context_gene # Start with row context

                    # If gene still "Unknown" from row context, try column to the left
                    if probable_gene == "Unknown" and c_idx > 0:
                        gene_cand_text_left = str(df.iloc[r_idx, c_idx - 1]).strip()
                        if gene_cand_text_left and gene_cand_text_left.lower() != 'nan':
                            temp_bases_left, _, _ = extract_sequence_and_direction_from_cell(gene_cand_text_left)
                            if not (temp_bases_left and is_valid_primer_sequence(temp_bases_left, min_len=15)):
                                cleaned_name_left = clean_gene_name(gene_cand_text_left)
                                if cleaned_name_left != "Unknown": probable_gene = cleaned_name_left
                    
                    # If gene is still "Unknown" and current cell has gene-like prefix before primer
                    if probable_gene == "Unknown" or len(probable_gene) < 2:
                        gene_part_match = re.match(r"([a-zA-Z0-9_.\-]+(?:[-/][a-zA-Z0-9_.\-]+)*)\s*[:\-(]", original_cell_text)
                        if gene_part_match:
                            potential_gene_prefix = clean_gene_name(gene_part_match.group(1))
                            if potential_gene_prefix != "Unknown": probable_gene = potential_gene_prefix
                    
                    # Use column header for direction if not found in cell
                    if probable_direction == "Unknown" and c_idx < len(df_headers_lower):
                        col_header_text = df_headers_lower[c_idx]
                        if any(k in col_header_text for k in FORWARD_KEYWORDS):
                            probable_direction = "Forward"
                        elif any(k in col_header_text for k in REVERSE_KEYWORDS):
                            probable_direction = "Reverse"
                    
                    # If gene is still "Unknown", check current column header (if not a direction/sequence keyword itself)
                    if probable_gene == "Unknown" and c_idx < len(df_headers_lower):
                        header_text = df_headers_lower[c_idx]
                        is_dir_header = any(k in header_text for k in FORWARD_KEYWORDS + REVERSE_KEYWORDS)
                        is_seq_header = any(k in header_text for k in ['sequence', 'primer', 'oligo', 'probe'])
                        if not is_dir_header and not is_seq_header and len(df_headers_original_case[c_idx]) > 1 : # Use original case for clean_gene_name
                            cleaned_header_gene = clean_gene_name(df_headers_original_case[c_idx])
                            if cleaned_header_gene != "Unknown": probable_gene = cleaned_header_gene

                    found_primers_in_df.append({
                        "PMCID": pmcid, "Gene": probable_gene, "Sequence": sequence,
                        "Orientation": probable_direction, "Source File": source_filename_context, # Use the context string
                        "Page": source_description, # General description (e.g. "XML Table", "PDF Page X")
                        "Original Cell Text": original_cell_text
                    })
            except Exception: # Catch errors within cell processing
                continue
    return found_primers_in_df


def extract_primers_from_xml(xml_path: Path, pmcid: str, hgnc_symbols: set) -> List[Dict]:
    found_xml_primers = []
    if not xml_path.exists(): return found_xml_primers

    try:
        with open(xml_path, 'rb') as f: xml_content_bytes = f.read()
        parser = etree.XMLParser(recover=True, strip_cdata=True, resolve_entities=False, no_network=True)
        root = etree.fromstring(xml_content_bytes, parser=parser)
    except Exception as e:
        print(f"  Error parsing XML for {pmcid} ({xml_path.name}): {e}")
        return found_xml_primers

    # --- 1. Process Tables within XML ---
    table_elements = root.xpath("//table[ancestor::table-wrap]") # More specific: tables within table-wraps
    if not table_elements: # Fallback to any table element
        table_elements = root.xpath("//table")
    
    print(f"  Found {len(table_elements)} <table> elements in XML for {pmcid}.")
    for idx, table_el in enumerate(table_elements):
        # print(f"    Processing XML Table {idx+1} in {pmcid}...")
        df_from_xml = parse_xml_table_to_dataframe(table_el)
        if not df_from_xml.empty:
            source_desc = f"XML Table {idx+1} (cols: {list(df_from_xml.columns)})"
            primers_from_table = scan_dataframe_for_primers(df_from_xml, pmcid, source_desc, xml_path.name, hgnc_symbols)
            if primers_from_table:
                found_xml_primers.extend(primers_from_table)
        # else:
            # print(f"      XML Table {idx+1} parsed to empty DataFrame.")

    # --- 2. Process Prose Text ---
    # Define XPaths for prose content, trying to exclude table cell content already processed
    prose_xpaths = [
        "//sec[translate(@sec-type,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz')='methods']//p[not(.//table)]",
        "//sec[translate(@sec-type,'ABCDEFGHIJKLMNOPQRSTUVWXYZ','abcdefghijklmnopqrstuvwxyz')='materialsmethods']//p[not(.//table)]",
        "//body//p[not(ancestor::table-wrap) and not(ancestor::fig) and not(.//table)]",
        "//fig//caption//p[not(.//table)]",
        "//table-wrap//caption//p[not(.//table)]", # Caption of table-wrap if it doesn't contain a table itself
        "//table-wrap-foot//p[not(.//table)]"
    ]
    unique_prose_blocks = set()
    dna_candidate_regex = re.compile(r'\b((?:[ACGT]\s*){15,50})\b', re.IGNORECASE)

    for xp in prose_xpaths:
        try:
            elements = root.xpath(xp)
            for elem in elements:
                text_block = get_element_text_content(elem)
                if not text_block or len(text_block) < 20 or text_block in unique_prose_blocks:
                    continue
                unique_prose_blocks.add(text_block)

                for match in dna_candidate_regex.finditer(text_block):
                    raw_dna_match_with_spaces = match.group(1)
                    core_bases, direction_from_text_body, _ = \
                        extract_sequence_and_direction_from_cell(raw_dna_match_with_spaces) # Use this to clean and get initial direction

                    if core_bases and is_valid_primer_sequence(core_bases):
                        sequence = core_bases
                        probable_direction = direction_from_text_body if direction_from_text_body else "Unknown"
                        probable_gene = "Unknown"
                        
                        match_start_in_block = match.start()
                        match_end_in_block = match.end()
                        
                        # Context for direction and gene (search around the raw_dna_match_with_spaces)
                        context_chars = 80 # How many chars before/after for context
                        text_before = text_block[max(0, match_start_in_block - context_chars) : match_start_in_block]
                        text_after = text_block[match_end_in_block : min(len(text_block), match_end_in_block + context_chars)]
                        
                        # Refine direction from local context if not found by extract_sequence_and_direction_from_cell
                        if probable_direction == "Unknown":
                            if any(k in text_before.lower() or k in text_after.lower() for k in FORWARD_KEYWORDS):
                                probable_direction = "Forward"
                            elif any(k in text_before.lower() or k in text_after.lower() for k in REVERSE_KEYWORDS):
                                probable_direction = "Reverse"
                        
                        # Gene finding in prose (simplified example, can be improved)
                        # Look for capitalized words or HGNC symbols near the primer
                        gene_search_context = text_before + raw_dna_match_with_spaces + text_after
                        potential_genes = re.findall(r'\b([A-Z][A-Za-z0-9-]{2,15})\b', gene_search_context) # Find capitalized words
                        best_dist = float('inf')
                        
                        common_prose_noise = {"PCR", "DNA", "RNA", "PRIMER", "FORWARD", "REVERSE", "SEQUENCE", "METHOD", "FIGURE", "TABLE"} | set(k.upper() for k in FORWARD_KEYWORDS) | set(k.upper() for k in REVERSE_KEYWORDS)

                        for pg_cand in potential_genes:
                            pg_upper = pg_cand.upper()
                            if pg_upper in hgnc_symbols and pg_upper not in common_prose_noise:
                                # Find position of pg_cand relative to raw_dna_match_with_spaces in gene_search_context
                                for m_pg in re.finditer(re.escape(pg_cand), gene_search_context, re.IGNORECASE):
                                    # Relative start of primer in gene_search_context
                                    primer_rel_start = len(text_before) 
                                    dist = abs(m_pg.start() - primer_rel_start)
                                    if dist < best_dist and dist < context_chars : # Must be close
                                        best_dist = dist
                                        probable_gene = clean_gene_name(pg_cand)
                        
                        original_context_snip = f"...{text_block[max(0,match_start_in_block-30):min(len(text_block),match_end_in_block+30)]}..."
                        found_xml_primers.append({
                            "PMCID": pmcid, "Gene": probable_gene, "Sequence": sequence,
                            "Orientation": probable_direction, "Source File": xml_path.name, 
                            "Page": "Main Text Prose", "Original Cell Text": original_context_snip
                        })
        except Exception: continue # Ignore errors for a single XPath or element
            
    return found_xml_primers


def find_primers_sequence_first(pdf_path, page_spec="all", hgnc_symbols: set = set()):
    all_found_primers_details = [] # This will store dicts from scan_dataframe_for_primers
    pdf_filename = os.path.basename(pdf_path)
    
    # Timeout for each Camelot call (lattice and stream)
    CAMELOT_CALL_TIMEOUT = 60  # Seconds (e.g., 1.5 minutes per Camelot flavor call)

    # print(f"Attempting to read tables from '{pdf_filename}' pages: {page_spec}") # Already printed in main
    
    start_camelot_time = time.time()
    lattice_tables_content = []
    stream_tables_content = []

    # Camelot parameters
    camelot_params_lattice = {
        'pages': page_spec, 
        'flavor': 'lattice', 
        'line_scale': 30, 
        'shift_text': [' '], 
        'copy_text': ['v'], 
        'suppress_stdout': True
    }
    camelot_params_stream = {
        'pages': page_spec, 
        'flavor': 'stream', 
        'suppress_stdout': True
    }

    with ProcessPool(max_workers=1) as pool: # Using 1 worker as we do calls sequentially
        # Lattice
        try:
            # print(f"  Submitting 'lattice' task for {pdf_filename}...")
            future_lattice = pool.schedule(camelot.read_pdf, args=[pdf_path], kwargs=camelot_params_lattice, timeout=CAMELOT_CALL_TIMEOUT)
            tables_lattice_obj = future_lattice.result()  # Blocks until result or timeout
            if tables_lattice_obj and tables_lattice_obj.n > 0:
                lattice_tables_content = list(tables_lattice_obj)
            # print(f"  Lattice found {len(lattice_tables_content)} table(s).")
        except PebbleTimeoutError:
            print(f"  Lattice flavor TIMED OUT for {pdf_filename} after {CAMELOT_CALL_TIMEOUT}s on pages '{page_spec}'.")
        except Exception as e_lattice:
            print(f"  Lattice flavor failed for {pdf_filename} (pages '{page_spec}'). Error: {e_lattice}")

        # Stream
        try:
            # print(f"  Submitting 'stream' task for {pdf_filename}...")
            future_stream = pool.schedule(camelot.read_pdf, args=[pdf_path], kwargs=camelot_params_stream, timeout=CAMELOT_CALL_TIMEOUT)
            tables_stream_obj = future_stream.result() # Blocks until result or timeout
            if tables_stream_obj and tables_stream_obj.n > 0:
                stream_tables_content = list(tables_stream_obj)
            # print(f"  Stream found {len(stream_tables_content)} table(s).")
        except PebbleTimeoutError:
            print(f"  Stream flavor TIMED OUT for {pdf_filename} after {CAMELOT_CALL_TIMEOUT}s on pages '{page_spec}'.")
        except Exception as e_stream:
            print(f"  Stream flavor failed for {pdf_filename} (pages '{page_spec}'). Error: {e_stream}")
    
    end_camelot_time = time.time()
    print(f"  Camelot PDF processing for pages '{page_spec}' took: {end_camelot_time - start_camelot_time:.2f} seconds (including any timeouts).")
        
    combined_tables_list = lattice_tables_content + stream_tables_content
        
    if not combined_tables_list:
        # print(f"  No tables found by Camelot in {pdf_filename} on pages '{page_spec}'.") # Already handled by individual prints
        return [] # Return empty list, not all_found_primers_details which isn't defined yet in this scope
        
    # print(f"  Found {len(combined_tables_list)} potential table regions in {pdf_filename}.")

    start_python_processing_time = time.time()
    # --- (The rest of your table scanning, cell iteration, and context gathering logic) ---
    # This part iterates `combined_tables_list` and calls `scan_dataframe_for_primers`
    # or directly processes df from table_obj to populate `all_found_primers_details`
    
    unique_table_identifiers = set()
    for table_idx, table_obj in enumerate(combined_tables_list):
        try:
            table_content_hash = hash(table_obj.df.to_string())
            table_id = (table_obj.page, table_content_hash, table_obj.flavor)
            if table_id in unique_table_identifiers: continue
            unique_table_identifiers.add(table_id)
        except Exception: pass
        
        df_from_pdf = table_obj.df
        if df_from_pdf.empty: continue
        
        source_desc = f"PDF Page {table_obj.page}, Table {table_idx+1} ({table_obj.flavor})"
        # Extract PMCID from the PDF filename to pass to scan_dataframe_for_primers
        current_pmcid = extract_pmcid_from_filename(pdf_filename)

        # Ensure scan_dataframe_for_primers is defined and accessible
        primers_from_pdf_table = scan_dataframe_for_primers(
            df_from_pdf, 
            current_pmcid, 
            source_desc,
            pdf_filename, # filename_or_context
            hgnc_symbols
        )
        if primers_from_pdf_table:
            all_found_primers_details.extend(primers_from_pdf_table)

    end_python_processing_time = time.time()
    # print(f"  Python post-Camelot processing took: {end_python_processing_time - start_python_processing_time:.2f} seconds.")
            
    # The `all_found_primers_details` list contains dictionaries already in the desired final format
    # as produced by `scan_dataframe_for_primers`.
    # No further transformation loop is strictly needed here if `scan_dataframe_for_primers`
    # correctly sets all required keys ("PMCID", "Gene", "Orientation", etc.).
    # The error you had previously (KeyError: 'Probable Gene') would be in the loop below if it existed.
    # The `scan_dataframe_for_primers` should be outputting with "Gene" and "Orientation".
    
    # For clarity, let's assume scan_dataframe_for_primers returns the final desired structure:
    # {"PMCID": ..., "Gene": ..., "Sequence": ..., "Orientation": ..., "Source File": ..., "Page": ..., "Original Cell Text": ...}
    
    return all_found_primers_details


# ###################################################################################
# # --- (Main Execution Block - if __name__ == "__main__": ...) ---
# ###################################################################################
if __name__ == "__main__":
    # --- 0. Configuration ---
    records_to_fetch = 500 # Example from your code
    START_YEAR = 2025
    END_YEAR_PROCESSING = 2015 # Example from your code
    N_LAST_PAGES = 5 # Used for PDF page targeting

    print("Loading HGNC symbols...")
    HGNC = load_hgnc_symbol_set()
    # xml_paths = [] # This isn't strictly necessary if processing XML immediately

    # --- Main Year Loop ---
    for year_to_process in range(START_YEAR, END_YEAR_PROCESSING - 1, -1): # Use year_to_process
        print(f"\n{'='*60}")
        print(f"STARTING PROCESSING FOR YEAR: {year_to_process}")
        print(f"{'='*60}\n")

        master_primer_list_for_year = [] # Initialize for the current year's results

        # --- 1. Define year-specific query for Europe PMC ---
        base_query = 'OPEN_ACCESS:y AND HAS_FT:y AND (METHODS:"qPCR" OR "RT-PCR" OR "real time PCR") AND (ABSTRACT:"pluripotent" OR "iPSC" OR "PSC" OR "hPSC" OR "ESC" OR "hESC")'
        query = f"{base_query} AND (FIRST_PDATE:{year_to_process})"
        print(f"Searching Europe PMC with query: {query}")
        
        epmc_hits_for_year = []
        try:
            epmc_hits_for_year = europepmc_search_all(query, max_records=records_to_fetch)
            print(f"Retrieved {len(epmc_hits_for_year)} records for {year_to_process}.")
        except Exception as e_search:
            print(f"Could not complete search for year {year_to_process}. Error: {e_search}")
            continue

        if not epmc_hits_for_year:
            print(f"No records found for {year_to_process}. Skipping to next year.")
            continue

        # --- 2. Process each paper for this year ---
        for hit_index, hit in enumerate(tqdm(epmc_hits_for_year, desc=f"Processing Papers for {year_to_process}")):
            current_pmcid_str = None
            if isinstance(hit, dict):
                current_pmcid_str = hit.get("pmcid") or hit.get("id")
            else:
                current_pmcid_str = str(hit)
            
            if not (current_pmcid_str and current_pmcid_str.startswith("PMC")):
                continue
            
            print(f"\n  Processing PMCID: {current_pmcid_str} ({hit_index+1}/{len(epmc_hits_for_year)}) for year {year_to_process}")
            
            primers_found_for_this_pmcid_count = 0 # Initialize for THIS PMCID

            try:
                xml_file_for_current_paper = get_or_download_xml(current_pmcid_str, DATA_DIR)
                
                # --- 2c. Read primers from main text XML ---
                if xml_file_for_current_paper.exists():
                    # print(f"    Extracting primers from XML: {xml_file_for_current_paper.name}")
                    xml_primers = extract_primers_from_xml(xml_file_for_current_paper, current_pmcid_str, HGNC)
                    if xml_primers:
                        master_primer_list_for_year.extend(xml_primers)
                        primers_found_for_this_pmcid_count += len(xml_primers)
                    # print(f"      Found {len(xml_primers)} primer items from XML. Total for {current_pmcid_str}: {primers_found_for_this_pmcid_count}")

                # --- 2d. Fetch, Rename, and Process Supplements for THIS PMCID ---
                renamed_supp_files_for_this_pmcid = [] # Initialize to ensure it's always defined

                if primers_found_for_this_pmcid_count > 8:
                    print(f"      Sufficient primers ({primers_found_for_this_pmcid_count}) found for {current_pmcid_str} from XML. Skipping its supplement processing.")
                else:
                    # print(f"    Fetching supplements for {current_pmcid_str} (XML found: {primers_found_for_this_pmcid_count})...")
                    downloaded_supp_files = fetch_supplements_from_xml(current_pmcid_str, xml_file_for_current_paper, DATA_DIR / "supp")
                    
                    if downloaded_supp_files:
                        renamed_supp_files_for_this_pmcid = rename_supp_files(current_pmcid_str, downloaded_supp_files)
                        # print(f"      ↳ {len(renamed_supp_files_for_this_pmcid)} supplement(s) to process for {current_pmcid_str}.")
                    # else:
                        # print(f"      No supplements found or downloaded for {current_pmcid_str}.")

                # Process the collected supplements for THIS PMCID
                if renamed_supp_files_for_this_pmcid: # Only loop if there are supplements and we haven't skipped due to XML
                    for supp_file_path_obj in tqdm(renamed_supp_files_for_this_pmcid, desc=f"Supps for {current_pmcid_str}", leave=False):
                        if primers_found_for_this_pmcid_count > 8:
                            print(f"      Sufficient primers ({primers_found_for_this_pmcid_count}) now found for {current_pmcid_str}. Skipping its remaining supplements.")
                            break # Correctly breaks from THIS PMCID's supplement loop

                        supp_file_path_str = str(supp_file_path_obj)
                        supp_filename_for_log = os.path.basename(supp_file_path_str)
                        # print(f"\n        Processing Supplement File: {supp_filename_for_log}")
                        
                        extracted_supp_primers = []
                        page_spec_for_camelot = "all" 
                        process_this_file_fully = True # Renamed from process_this_pdf_fully

                        # Determine page_spec and if it's a reporting summary (for PDFs)
                        if supp_filename_for_log.lower().endswith(".pdf"):
                            try:
                                with pdfplumber.open(supp_file_path_str) as pdf_doc:
                                    if not pdf_doc.pages:
                                        process_this_file_fully = False
                                    else:
                                        total_pages = len(pdf_doc.pages)
                                        first_page_text = pdf_doc.pages[0].extract_text() if pdf_doc.pages[0] else ""
                                        if first_page_text and "reporting summary" in first_page_text.lower():
                                            page_spec_for_camelot = "1"
                                        elif total_pages == 0: page_spec_for_camelot = "1"
                                        elif total_pages <= N_LAST_PAGES: page_spec_for_camelot = f"1-{total_pages}"
                                        else:
                                            start_page = total_pages - N_LAST_PAGES + 1
                                            page_spec_for_camelot = f"{start_page}-{total_pages}"
                            except Exception as e_pg_cnt:
                                # print(f"        Error getting page count for {supp_filename_for_log}: {e_pg_cnt}. Defaulting to all pages.")
                                page_spec_for_camelot = "all" # Keep process_this_file_fully as True to attempt parsing

                        # Process the file based on its type
                        if process_this_file_fully:
                            if supp_filename_for_log.lower().endswith(".pdf"):
                                extracted_supp_primers = find_primers_sequence_first(supp_file_path_str, page_spec=page_spec_for_camelot, hgnc_symbols=HGNC)
                            elif supp_filename_for_log.lower().endswith(".docx"):
                                extracted_supp_primers = extract_primers_from_docx(supp_file_path_obj, current_pmcid_str, HGNC)
                            # Add more elif for other file types if needed

                        if extracted_supp_primers:
                            master_primer_list_for_year.extend(extracted_supp_primers)
                            primers_found_for_this_pmcid_count += len(extracted_supp_primers)
                        # print(f"        Finished {supp_filename_for_log}. Found {len(extracted_supp_primers)}. Total for PMCID {current_pmcid_str}: {primers_found_for_this_pmcid_count}")
            
            except requests.HTTPError as e:
                print(f"    ⚠️ HTTPError for {current_pmcid_str} (XML or Supplement download): {e}")
            except Exception as e_paper_processing: # This catches other errors during THIS paper's processing
                print(f"    ⚠️ Unexpected error processing paper {current_pmcid_str}: {e_paper_processing}")
                import traceback
                traceback.print_exc() # This will show where the NameError occurred if it's within this try
                # Continue to the next paper
            
            time.sleep(0.3) # Politeness delay for Europe PMC
        # --- End of loop for individual papers (epmc_hits_for_year) ---

        # --- Save results for the CURRENT YEAR ---
        if master_primer_list_for_year:
            print(f"\n--- Consolidating and Saving Results for YEAR {year_to_process} ---")
            # ... (rest of your CSV saving logic for the year, which seems correct) ...
            final_df_year = pd.DataFrame(master_primer_list_for_year)
            if not final_df_year.empty:
                expected_columns = ['PMCID', 'Gene', 'Sequence', 'Orientation', 'Source File', 'Page', 'Original Cell Text']
                actual_columns_in_df = final_df_year.columns.tolist()
                subset_for_duplicates = [col for col in expected_columns if col in actual_columns_in_df]
                if not subset_for_duplicates: final_df_year.drop_duplicates(inplace=True, keep='first')
                else: final_df_year.drop_duplicates(subset=subset_for_duplicates, inplace=True, keep='first')
                print(f"Total unique primer entries found for {year_to_process}: {len(final_df_year)}")
                # (Print and save logic)
                try:
                    output_dir = DATA_DIR / "yearly_results"; output_dir.mkdir(parents=True, exist_ok=True)
                    output_csv_path_year = output_dir / f"master_extracted_primers_{year_to_process}.csv"
                    final_df_year.to_csv(output_csv_path_year, index=False)
                    print(f"  ✅ Saved {len(final_df_year)} unique extracted primers for {year_to_process} to {output_csv_path_year}")
                except Exception as e_csv: print(f"  ❌ Error saving CSV for {year_to_process}: {e_csv}")
            else: print(f"No primer data to save for {year_to_process} (DataFrame empty).")
        else: print(f"No primers extracted for YEAR {year_to_process}.")
        print(f"###### FINISHED PROCESSING FOR YEAR: {year_to_process} ######")

    print("\n\nBatch processing complete for all specified years.")

Loading HGNC symbols...
Loaded 193359 HGNC symbols.

STARTING PROCESSING FOR YEAR: 2025

Searching Europe PMC with query: OPEN_ACCESS:y AND HAS_FT:y AND (METHODS:"qPCR" OR "RT-PCR" OR "real time PCR") AND (ABSTRACT:"pluripotent" OR "iPSC" OR "PSC" OR "hPSC" OR "ESC" OR "hESC") AND (FIRST_PDATE:2025)
Retrieved 500 records for 2025.


Processing Papers for 2025:   0%|                                                      | 0/500 [00:00<?, ?it/s]


  Processing PMCID: PMC12076839 (1/500) for year 2025
  Found 0 <table> elements in XML for PMC12076839.



Supps for PMC12076839:  50%|██████████████████████████▌                          | 1/2 [00:00<00:00,  1.48it/s]
                                                                                                               

      Sufficient primers (42) now found for PMC12076839. Skipping its remaining supplements.


Processing Papers for 2025:   0%|                                            | 1/500 [00:09<1:20:37,  9.69s/it]


  Processing PMCID: PMC12008852 (2/500) for year 2025
  Found 0 <table> elements in XML for PMC12008852.


Processing Papers for 2025:   0%|                                            | 1/500 [00:45<6:15:52, 45.19s/it]


KeyboardInterrupt: 

In [4]:
import pandas as pd
import glob
import os
from pathlib import Path
from collections import Counter # Import Counter

def aggregate_primer_data(input_dir: Path, output_file: Path):
    """
    Reads all 'master_extracted_primers_YYYY.csv' files from the input directory,
    aggregates primer sequences, counts occurrences, lists source PMCIDs,
    Orientations, and Genes (with counts), and saves the result to a new CSV file.

    Args:
        input_dir (Path): The directory containing the yearly CSV files.
        output_file (Path): The path to save the aggregated CSV file.
    """
    print(f"Looking for yearly CSV files in: {input_dir}")

    csv_files = glob.glob(str(input_dir / "master_extracted_primers_*.csv"))

    if not csv_files:
        print(f"No CSV files found in {input_dir} matching the pattern 'master_extracted_primers_*.csv'.")
        print("Please ensure your yearly CSV files are in that directory and named correctly.")
        return

    print(f"Found {len(csv_files)} CSV files to process:")
    for f in csv_files:
        print(f"  - {os.path.basename(f)}")

    all_dataframes = []
    required_columns = ['Sequence', 'PMCID', 'Orientation', 'Gene']

    for file_path in csv_files:
        try:
            df = pd.read_csv(file_path)
            missing_cols = [col for col in required_columns if col not in df.columns]
            if missing_cols:
                print(f"Warning: Skipping file {os.path.basename(file_path)} as it's missing column(s): {', '.join(missing_cols)}.")
                continue
            all_dataframes.append(df)
        except pd.errors.EmptyDataError:
            print(f"Warning: Skipping empty file {os.path.basename(file_path)}.")
        except Exception as e:
            print(f"Error reading {os.path.basename(file_path)}: {e}")

    if not all_dataframes:
        print("No data loaded from CSV files. Aggregation cannot proceed.")
        return

    master_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"\nTotal rows loaded from all CSVs: {len(master_df)}")

    if master_df.empty:
        print("The combined dataframe is empty. Nothing to aggregate.")
        return

    # Standardize and clean data
    master_df['Sequence'] = master_df['Sequence'].astype(str).str.upper()
    master_df['PMCID'] = master_df['PMCID'].astype(str)
    master_df['Orientation'] = master_df['Orientation'].astype(str).str.strip().fillna('Unknown')
    master_df['Gene'] = master_df['Gene'].astype(str).str.strip().fillna('Unknown')
    
    master_df['Orientation'] = master_df['Orientation'].apply(lambda x: 'Unknown' if x == '' else x)
    master_df['Gene'] = master_df['Gene'].apply(lambda x: 'Unknown' if x == '' else x)

    master_df.dropna(subset=['Sequence', 'PMCID'], inplace=True)
    master_df = master_df[master_df['Sequence'] != '']

    print(f"Processing {len(master_df)} rows for aggregation...")

    # --- MODIFIED AGGREGATION FOR GENES ---
    def aggregate_gene_counts(series):
        # Count occurrences of each gene in the series
        counts = Counter(str(g) for g in series)
        # Sort by count (descending), then by gene name (ascending) for tie-breaking
        sorted_counts = sorted(counts.items(), key=lambda item: (-item[1], item[0]))
        # Format as "GeneName (count)"
        return ', '.join([f"{gene} ({count})" for gene, count in sorted_counts])

    aggregated_data = master_df.groupby('Sequence').agg(
        Occurrence_Count=('Sequence', 'size'),
        Source_PMCIDs=('PMCID', lambda x: ', '.join(sorted(list(set(str(i) for i in x))))),
        Source_Orientations=('Orientation', lambda x: ', '.join(sorted(list(set(str(i) for i in x))))),
        Source_Genes=('Gene', aggregate_gene_counts) # Use the new aggregation function
    ).reset_index()
    # --- END MODIFICATION ---

    aggregated_data = aggregated_data.sort_values(by='Occurrence_Count', ascending=False)

    print(f"\nAggregation complete. Found {len(aggregated_data)} unique primer sequences.")

    try:
        output_file.parent.mkdir(parents=True, exist_ok=True)
        aggregated_data.to_csv(output_file, index=False)
        print(f"\nSuccessfully saved aggregated primer data to: {output_file}")
    except Exception as e:
        print(f"\nError saving aggregated data to {output_file}: {e}")

if __name__ == "__main__":
    INPUT_DATA_DIR = Path("data/psc/yearly_results") 
    AGGREGATED_OUTPUT_FILE = INPUT_DATA_DIR / "aggregated_primer_summary_with_details.csv"

    aggregate_primer_data(INPUT_DATA_DIR, AGGREGATED_OUTPUT_FILE)

    print("\n--- Script Finished ---")
    # Example of how to quickly view the top results if needed:
    # if AGGREGATED_OUTPUT_FILE.exists():
    #     top_results_df = pd.read_csv(AGGREGATED_OUTPUT_FILE)
    #     print("\nTop 10 most frequent primers:")
    #     print(top_results_df.head(10).to_string())


Looking for yearly CSV files in: data\psc\yearly_results
Found 4 CSV files to process:
  - master_extracted_primers_2022.csv
  - master_extracted_primers_2023.csv
  - master_extracted_primers_2024.csv
  - master_extracted_primers_2025.csv

Total rows loaded from all CSVs: 6522
Processing 6522 rows for aggregation...

Aggregation complete. Found 4751 unique primer sequences.

Successfully saved aggregated primer data to: data\psc\yearly_results\aggregated_primer_summary_with_details.csv

--- Script Finished ---
